<a href="https://colab.research.google.com/github/Droslj/scATAC-seq-complete-/blob/Google-colab/scATAC_seq_(1)_scVI_exec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

scATAC seq, based on scATAC seq processing Galaxy tutorials (scATAC preprocessing (2), Standard scATAC seq processing pipeline (1))
Raw AD Objects created in Galaxy using customized Galaxy WF with Snapatac2 and imported
Sources:
(1) https://usegalaxy.eu/training-material/topics/single-cell/tutorials/scatac-preprocessing-tenx/tutorial.html#mapping-reads-to-a-reference-genome, (2) https://usegalaxy.eu/training-material/topics/single-cell/tutorials/scatac-standard-processing-snapatac2/tutorial.html
Data taken from the following NCBI study:
(3) Metabolic adaptation pilots the differentiation of human hematopoietic cells (https://www.ncbi.nlm.nih.gov/bioproject/PRJNA1015713)

Import preprocessed Anndata object for two biological replicates, SRR26046013 (cells treated with AOA inhibitor), and SRR26046019 (untreated cells).
Raw AD objects preprocessed as follows:
(1) Import matrices
(2) Compute fragment size distribution
(3) Compute TSS enrichment
(4) Filter cell counts based on TSSe
(5) Create cell by bin matrix based on 500 bp wide bins accross the whole genome
(6) Preprocessed for scvi model

This file focuses on DA analysis using scvi package.

In [1]:
!pip install snapatac2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.2/687.2 kB 45.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 14.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into accoun

In [2]:
!pip install snapatac2 -q

In [3]:
!pip show snapatac2

Name: snapatac2
Version: 2.8.0
Summary: SnapATAC2: Single-cell epigenomics analysis pipeline
Home-page: https://github.com/
Author: Kai Zhang <kai@kzhang.org>
Author-email: Kai Zhang <zhangkai33@westlake.edu.cn>
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: anndata, igraph, kaleido, macs3, multiprocess, natsort, numpy, pandas, plotly, polars, pooch, pyarrow, pyfaidx, rustworkx, scikit-learn, scipy, tqdm, typeguard
Required-by: 


In [4]:
import snapatac2 as snap

In [5]:
!pip install umap-learn -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 6.6 MB/s eta 0:00:00


In [6]:
import umap.umap_ as umap


In [7]:
from umap import UMAP

In [8]:
!pip install scanpy -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 9.9 MB/s eta 0:00:00


In [9]:
import scanpy as sc

In [10]:
!pip install harmonypy -q

In [11]:
import scanpy.external as sce

In [12]:
pip show scanpy

Name: scanpy
Version: 1.10.4
Summary: Single-Cell Analysis in Python.
Home-page: https://scanpy.org
Author: Alex Wolf, Philipp Angerer, Fidel Ramirez, Isaac Virshup, Sergei Rybakov, Gokcen Eraslan, Tom White, Malte Luecken, Davide Cittaro, Tobias Callies, Marius Lange, Andrés R. Muñoz-Rojas
Author-email: 
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anndata, h5py, joblib, legacy-api-wrap, matplotlib, natsort, networkx, numba, numpy, packaging, pandas, patsy, pynndescent, scikit-learn, scipy, seaborn, session-info, statsmodels, tqdm, umap-learn
Required-by: 


In [13]:
!pip install tqdm -q

In [14]:
import numpy as np

In [15]:
import anndata as ad

In [16]:
import matplotlib.pyplot as plt

In [17]:
import seaborn as sns

In [18]:
import plotly.subplots as sp
import plotly.graph_objects as go

In [19]:
from scipy import stats

In [20]:
import pandas as pd

In [21]:
from tqdm.notebook import tqdm

In [22]:
!pip install --quiet scvi-tools

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.3/459.3 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.1/356.1 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 49.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all t

In [23]:
!pip install --quiet scvi-tools

In [24]:
!pip3 install leidenalg -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.7 MB/s eta 0:00:00


In [25]:
!pip install harmonypy -q  # Make sure harmonypy is installed

In [26]:
import scanpy.external as sce

In [27]:
import scvi

In [28]:
import os

In [29]:
import pooch

In [30]:
import scvi

In [31]:
import torch

In [ ]:
import os

In [ ]:
import tempfile

In [ ]:
import shutil

# Import reads from google drive, preprocessed for scATAC-seq

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
adata = sc.read_h5ad('/content/drive/MyDrive/Colab Notebooks/AD_combined_scvi.h5ad')

# Preprocess AD object for scvi

In [ ]:
# Filter out peaks that are rarely detected, to make the model train faster:
print("# regions before filtering:", adata.shape[-1])

In [ ]:
# compute the threshold: 5% of the cells
min_cells = int(adata.shape[0] * 0.05)

In [ ]:
# in-place filtering of regions
sc.pp.filter_genes(adata, min_cells=min_cells)

print("# regions after filtering:", adata.shape[-1])

# Set up, train and save/load the model

In [ ]:
#scvi.model.PEAKVI.setup_anndata(adata)

In [ ]:
#model = scvi.model.PEAKVI(adata)
#model.train()

In [ ]:
# Create a temporary directory if save_dir is not defined
save_dir = tempfile.TemporaryDirectory()

model_dir = os.path.join(save_dir.name, "peakvi_pbmc")
model.save(model_dir, overwrite=True)

# Copy the model directory to your Google Drive
drive_path = "/content/drive/MyDrive/Colab Notebooks/peakvi_pbmc"  # Choose your desired path
shutil.copytree(model_dir, drive_path)

# ... (rest of your code) ...

# Clean up the temporary directory
save_dir.cleanup()

print(f"Model saved to: {drive_path}")

In [ ]:
#Load the model
model = scvi.model.PEAKVI.load('/content/drive/MyDrive/Colab Notebooks/peakvi_pbmc', adata=adata)

In [ ]:
model

# Visualizing and analyzing the latent space

In [ ]:
PEAKVI_LATENT_KEY = "X_peakvi"

latent = model.get_latent_representation()
adata.obsm[PEAKVI_LATENT_KEY] = latent
latent.shape

In [ ]:
PEAKVI_CLUSTERS_KEY = "clusters_peakvi"

# compute the k-nearest-neighbor graph that is used in both clustering and umap algorithms
sc.pp.neighbors(adata, use_rep=PEAKVI_LATENT_KEY, bat)
# compute the umap
sc.tl.umap(adata, min_dist=0.2)
# cluster the space (we use a lower resolution to get fewer clusters than the default)
sc.tl.leiden(adata, key_added=PEAKVI_CLUSTERS_KEY, resolution=0.2)

In [ ]:
sc.pl.umap(adata, color=PEAKVI_CLUSTERS_KEY)

In [ ]:
adata.obs

# Differential accessibility

In [ ]:
# Get the unique cluster names
cluster_names = adata.obs[PEAKVI_CLUSTERS_KEY].unique()
print(cluster_names)

In [ ]:
# Choose the actual cluster names for group1 and group2
# For example, if your clusters are labeled '0', '1', '2', etc., you might use:
da_res11 = model.differential_accessibility(
    groupby=PEAKVI_CLUSTERS_KEY,
    group1=cluster_names[0],  # Replace with actual cluster name
    group2=cluster_names[1],  # Replace with actual cluster name
    two_sided=True,
    batch_correction=True
)

In [ ]:
da_res11